# KKRnano Tutorial


### Overview

* [Part 0: basic introduction to KKRnano](00_KKRnano_tutorial_basics.ipynb)
* [Part 1: starting KKRnano from Voronoi](01_KKRnano_tutorial_simple_calculation.ipynb)
* [Part 2: continuing KKRnano calculations](02_KKRnano_tutorial_continue_calculations.ipynb)
* [Part 3: use a KKRnano workflow](03_KKRnano_tutorial_DOSworkchain.ipynb)
* [Part 4: StrucWithPotData](04_KKRnano_tutorial_StrucWithPotData.ipynb)

## Part 4: use the StrucWithPotData input

In [ ]:
from aiida.orm import StructureData, load_node, CalcJobNode, Dict, RemoteData, StructureData, Bool, SinglefileData, Int, Float, Code
import numpy as np
from aiida import load_profile

_ = load_profile()

# 4. `StrucWithPotData` 

## **4.1 `StrucWithPotData` - starting from a structure with potential data**


The converted potentials files, which are retrieved, can be used e.g. to create a new structure by stacking preexisting structures and creating some defect containing supercell by replacing/adding some potential form another calculation with a different element. However, this  is in principle difficult as the Fermi levels of the individual potentials would have to align which they tend not to, if one mixes different calculations. A workaround that is to make use of the possibility to run CPA calculations with KKRhost. This way one can obtain a good starting potential with aligned Fermi levels.


To ensure the pronvenance of the input files, i. e. for good bookkeeping, it is helpful to store all the files together in a single object. This can be done using the `StrucWithPotData` datatype that comes with the plugin. Essentially this is just the fixed set of an AiiDA `StructureData` object a list of formatted potential files for the sites of the structure and a list of the shapes of the individual shapefunctions for the corresponding sites in the same order as the site list of the structure.

There are several ways how such a `StrucWithPotData` object can be created. The direct way is to provide the structure, the list of potential files, and the list of shapefunction files like so (here we use some files from the local file system, say from a run without AiiDA, bear in mind that shapes and potentials must match!):

In [ ]:
from aiida_kkr.data.strucwithpot import StrucWithPotData
from aiida.orm import SinglefileData
import os

#Preparing lists from existing file in tutorial subdirectory
path=os.getcwd()+"/example_shapes_pots/"
list1=[SinglefileData(path+"shape.0000001"),SinglefileData(path+"shape.0000002")]
list2=[SinglefileData(path+"vpot.00001"),SinglefileData(path+"vpot.00002")]

In [ ]:
from aiida.orm import StructureData
import numpy as np
alat = 3.61
bravais = alat*np.array([[1.0, 0.0, 0], [0.0, 1.0, 0], [0, 0, 1.0]])

Cu_bulk = StructureData(cell=bravais)
Cu_bulk.append_atom(position=[0,0,0], symbols='Cu')
Cu_bulk.append_atom(position=alat*np.array([0,0.5,0.5]), symbols='Cu')

StrucWithPotData(
    passedStructure=Cu_bulk,
    list_of_shapes=list1,
    list_of_pots=list2
)

The lists must consist of `SingleFileData` objects.


However, it is also possible to use a **KKRnano calculation convert calculation node** as an input to create a `StrucWithPotData` object:

In [ ]:
# stwpd_kkrnano=StrucWithPotData(
#     KKRnanoCalcNode=kkr_conv
# )
stwpd_kkrnano=load_node("7c85f241-621e-4634-8586-027b8d5914b9")
stwpd_kkrnano.store()

This traces back which `StructureData` object was used for the calculation (indirectly at some point via the Voronoi input, etc.), it finds a shapefunction, that must have been used at some point. However, ideally the shapes were written out as otherwise they (probably) do not match the potential files of the convert step (see remarks on Voronoi calculation). That is why this should always be set when running calculations aimed to be used in conjunction with the formatted potentials.
As the remote folder has to be accessed in the process of finding the shapes, this might firstly take a while, and secondly, this means that also the local folder is used temporarily to store the shapefunctions and potentials in before they are moved to the repository.
Also mind that only the convert calculation can be used as the regular run does not feature a formatted potential output. 

In addition also **KKRhost calculations** and **Voronoi calculation** can be read in and processed to `StructureData` objects. This is done similarly via
```
StrucWithPotData(
    KKRhostCalcNode=calc
)
```
or
```
StrucWithPotData(
    VoronoiCalcNode=calc
)
```
In our case:

In [ ]:
StrucWithPotData(
    VoronoiCalcNode=voro_calc
)

Here, it is actually irrelevant which argument one uses, as both internally trigger calls of the same functions. What is more, is that one also does need to worry about potential and shapes matching or some convert step, here, since these files are always present in the respective remote folder. Nonetheless, again, it is recommended to use the `RUNOPT=['WRITEALL']` in the Voronoi calculation to ensure that all shapes are written out, as KKRnano might have problems if it is supposed to implicitly use the same function for all sites.

## 4.2 Using CPA calculations as input
One can also print out the shapefunction or potential of the created `StrucWithPotData` object using `make_potential()` and `make_shapefun()`:

In [ ]:
print(stwpd_kkrnano.make_potential())

In [ ]:
print(stwpd_kkrnano.make_shapefun())

Moreover, one can get the sites as a `dict` using `site()` yielding a `list` with entries like this

In [ ]:
stwpd_kkrnano.sites()

### 4.2.1 Building a supercell from KKRhost CPA calculation

In KKRhost it is possible, as mentioned before to perform CPA calculations. Please refer to the documentation of KKRhost and the corresponding AiiDA plugin, to see how this is done.
The following example is a finished CPA calculation turned into a `StrucWithPotData` as described above. It is of Bi$_2$Te$_3$ with 0.1 antisite Te:

In [ ]:
CPA_calc=load_node("f6aea739-d2b2-4df1-8cb0-45c5fbbe6b3e")
CPA_calc.sites()

As can be seen from this, it is possible to have more than a single species at a site and having therefore multiple potential files. The input of the multiple potential files is still a list, so that one has to be careful to provide them in the right order, also as said list now is longer than the number of sites. However, this is simply circumvented by creating such a `StrucWithPotData` object directly from a KKRhost calculation as done for this example.

We will now build a supercell from this example. We first extract the potentials into lists using the `sites()` function. The second site will be the anti-site:

In [ ]:
Bi_pots=[CPA_calc.sites()[j]["potential"][0] for j in [0,1]] #checked 
anti_site_Bi_site_pots=[Bi_pots[0]]+[CPA_calc.sites()[1]["potential"][1]] #1st site Bi, 2nd site Te, checked
regular_Te_sites=[CPA_calc.sites()[j]["potential"] for j in range(2,5)] #checked

Then we load the structure, that was used for the CPA calulation, and one that the other was turned into by replacing the second Bi site with a Te site.

In [ ]:
#regular Bi2Te3 structure
Bi2Te3_struc=load_node("615d587e-0d7e-405f-b551-2238ef8c1928")
#anti-site Bi2Te3 structure
Bi2Te3_antisite_struc=load_node("2f523c7f-cc26-46e1-9b8a-44526c512fa9")

We now create different `StrucWithPotData` objects using the "correct" and the anti-site structures and potential lists.

In [ ]:
stwpd_CPA1=StrucWithPotData(passedStructure=Bi2Te3_struc,\
                            list_of_shapes=CPA_calc.shapes,\
                            list_of_pots=Bi_pots+regular_Te_sites)
stwpd_CPA_antisite=StrucWithPotData(passedStructure=Bi2Te3_antisite_struc,\
                                    list_of_shapes=CPA_calc.shapes,\
                            list_of_pots=Bi_pots+regular_Te_sites)

The we create our small "supercell" by stacking existing structures. When doing so, it is important that we make sure the order of the potential files stays correct.

In [ ]:


from rescale import stack_two_strucs

struc1=Bi2Te3_struc.get_pymatgen()
struc2=Bi2Te3_antisite_struc.get_pymatgen()


#stacking in b3 direction (countig starts at 0 for axes!)
#stack1 with antisite
newstruc1=stack_two_strucs(struc1,struc2)
pots1=stwpd_CPA1.potentials + stwpd_CPA_antisite.potentials
shapes1=stwpd_CPA1.shapes+stwpd_CPA_antisite.shapes



#stack2 without antisite
newstruc2=stack_two_strucs(struc1,struc1)
pots2=stwpd_CPA1.potentials+stwpd_CPA1.potentials
shapes2=stwpd_CPA1.shapes+ stwpd_CPA1.shapes

#stacking in b2 direction (countig starts at 0 for axes!)
#stack1 with antisite
newstruc3=stack_two_strucs(newstruc1,newstruc2, bravais_axis=1)
pots3=pots1+pots2
shapes3=shapes1+shapes2

In [ ]:
#Turning the "raw" data into a StrucWithPotData object
#we want to make sure that the primitive lattice constant is reused throughout the calculations.
#Hence, we specify this lattice constant
supercell=StrucWithPotData(passedStructure=StructureData(pymatgen=newstruc3), list_of_shapes=shapes3,\
                           list_of_pots=pots3,specified_lattice_constant=Float(struc1.lattice.c))

supercell.store()

The resulting structure can be plotted, and we see that in the lower right part a Bi atom is a Te atom.

In [ ]:
plot_kkr(StructureData(pymatgen=newstruc3),silent=True)

### 4.2.2 Starting a KKRano calculation with a `StrucWithPotData` object
Let's focus on starting a calculation using our `StrucWithPotData` object now.

In [ ]:
from aiida_kkr.calculations.kkrnano import KKRnanoCalculation

builder=KKRnanoCalculation.get_builder()

#Using suitable parameters
builder.parameters=load_node("ec6dab6f-a72e-4159-b5b6-df7da1a8fcff")
builder.metadata.options= {'resources': {'num_machines': 1, 'tot_num_mpiprocs': 40},\
                           'queue_name': 'th1-2020-64', 'withmpi': True}

builder.code = Code.get_from_string("KKRnanoVersuch2@iffslurm")

builder.strucwithpot=supercell

In [ ]:
#supercell_calc=submit(builder) #takes very long
supercell_calc=load_node("33804a10-be0f-4cfa-b73a-4b7de7eedbaf")
supercell_calc

In [ ]:
print("rms-error", supercell_calc.outputs.output_parameters["rms_all_iterations"][-5:])

One could now further investigate the implications of this by exploring the DOS, making larger supercells and studying the real space dependence of defects!